In [1]:
%pip install numpy matplotlib ipywidgets

Note: you may need to restart the kernel to use updated packages.


## Fundamentals of Analog-to-Digital Conversion (ADC)
### Objectives:
1. **Quantization:** Observe how bit depth affects signal fidelity and the "staircase" effect.
2. **SQNR:** Validate the $6.02N + 1.76$ dB rule.
3. **Aliasing:** Understand why the HackRF needs a specific sampling rate to see 16 MHz.

## 1. Interactive Quantization Exercise

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, VBox, HBox
from IPython.display import display, clear_output

def plot_adc(bits, signal_amp, adc_range, target_bw):
    # --- 1. Parameters ---
    fs = 32000
    t = np.arange(0, 0.01, 1/fs) 
    f_signal = 997
    
    # BW used for Process Gain calculation
    # Process Gain = 10 * log10( (fs/2) / (target_bw/2) )
    BW = target_bw
    
    # --- 2. Signal & Quantization ---
    signal = signal_amp * np.sin(2 * np.pi * f_signal * t)
    v_max, v_min = adc_range, -adc_range
    num_levels = 2**bits
    lsb = (v_max - v_min) / num_levels
    
    indices = np.floor((signal - v_min) / lsb)
    indices = np.clip(indices, 0, num_levels - 1)
    quantized = v_min + (indices + 0.5) * lsb
    error = signal - quantized
    
    # --- 3. Metrics ---
    sqnr_calc = 10 * np.log10(np.var(signal) / (np.var(error) + 1e-18))
    sqnr_theory = 6.02 * bits + 1.76
    
    # Process Gain: Ratio of Nyquist bandwidth to signal bandwidth
    # Limit BW to fs to avoid negative gain or math errors
    safe_bw = max(1, min(BW, fs))
    process_gain = 10 * np.log10(fs / safe_bw)
    sqnr_pg = sqnr_calc + process_gain
    
    # --- 4. Plotting ---
    clear_output(wait=True) 
    fig = plt.figure(figsize=(12, 10))
    
    # Subplot 1: Time Domain
    ax1 = plt.subplot(3, 1, 1)
    ax1.plot(t, signal, 'b--', alpha=0.4, label="Analog Input")
    ax1.step(t, quantized, where='mid', color='red', label="Digital Output")
    title_str = (f"LSB: {lsb:.4f} V | Theory: {sqnr_theory:.2f} dB | "
                 f"Calc: {sqnr_calc:.2f} dB | PG: {process_gain:.2f} dB | Total: {sqnr_pg:.2f} dB")
    ax1.set_title(title_str, fontsize=11, fontweight='bold')
    ax1.legend(loc='upper right')
    ax1.grid(True)
    
    # Subplot 2: Quantization Error
    ax2 = plt.subplot(3, 1, 2, sharex=ax1)
    ax2.plot(t, error, color='black', lw=0.8)
    ax2.set_ylabel("Error (V)")
    ax2.set_ylim(-adc_range, adc_range)
    ax2.grid(True)
    
    # Subplot 3: Frequency Domain
    ax3 = plt.subplot(3, 1, 3)
    n = len(quantized)
    freqs = np.fft.rfftfreq(n, 1/fs)
    fft_mag = (np.abs(np.fft.rfft(quantized))/(n/2) + 1e-10)
    ax3.plot(freqs, fft_mag, color='purple')
    
    # Visualizing the Bandwidth being analyzed
    ax3.axvspan(0, BW/2, color='green', alpha=0.1, label='Target BW (for PG)')
    ax3.set_title(f"Frequency Spectrum (Process Gain assumes filtering to {BW} Hz)")
    ax3.set_ylabel("Magnitude")
    ax3.set_xlabel("Frequency (Hz)")
    ax3.set_ylim(0, max(fft_mag)*1.2)
    ax3.legend()
    ax3.grid(True)
    
    plt.tight_layout()
    plt.show()

# --- UI Controls ---
# Using continuous_update=False to keep the interface snappy
bits_w = widgets.IntSlider(min=1, max=12, step=1, value=8, description='Bits', continuous_update=False)
amp_w = widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=2.0, description='Sig Amp', continuous_update=False)
range_w = widgets.FloatSlider(min=0.5, max=5.0, step=0.1, value=2.5, description='ADC Range', continuous_update=False)
bw_w = widgets.IntSlider(min=100, max=32000, step=100, value=16000, description='BW (Hz)', continuous_update=False)

# Map sliders to function arguments
out = interactive_output(plot_adc, {
    'bits': bits_w, 
    'signal_amp': amp_w, 
    'adc_range': range_w, 
    'target_bw': bw_w
})

# Display Layout
ui = VBox([
    HBox([bits_w, amp_w]),
    HBox([range_w, bw_w])
])
display(ui, out)

Output()

## 2. Aliasing & Nyquist

In [10]:
def plot_aliasing(f_signal):
    fs = 2000 # Fixed sampling rate at 2kHz (Nyquist limit = 1kHz)
    t_analog = np.linspace(0, 0.005, 1000)
    t_samples = np.arange(0, 0.005, 1/fs)
    
    sig_analog = 2.5 * np.sin(2 * np.pi * f_signal * t_analog)
    sig_samples = 2.5 * np.sin(2 * np.pi * f_signal * t_samples)
    
    plt.figure(figsize=(10, 4))
    plt.plot(t_analog, sig_analog, label="Real Signal", alpha=0.3)
    plt.stem(t_samples, sig_samples, 'r', label="ADC Samples")
    plt.plot(t_samples, sig_samples, 'r--', alpha=0.6, label="Perceived Signal")
    plt.title(f"Signal: {f_signal} Hz | Sampling Rate: {fs} Hz")
    plt.legend()
    plt.grid(True)
    plt.show()

print("Experiment: Increase frequency above 1000 Hz (Nyquist) and watch the 'Perceived Signal' slow down.")
interact(plot_aliasing, f_signal=widgets.IntSlider(min=100, max=1900, step=100, value=400))

Experiment: Increase frequency above 1000 Hz (Nyquist) and watch the 'Perceived Signal' slow down.


interactive(children=(IntSlider(value=400, description='f_signal', max=1900, min=100, step=100), Output()), _d…

<function __main__.plot_aliasing(f_signal)>

## 3. FFT Noise 

In [56]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, VBox, HBox
from IPython.display import display, clear_output

def plot_adc_final(bits, m_power):
    # --- 1. Parameters ---
    fs = 4096
    M = 2**m_power 
    t = np.arange(M) / fs
    f_main = 997
    signal_amp = 2.5
    adc_range = signal_amp
    
    # --- 2. Signal & Quantization ---
    signal = signal_amp * np.sin(2 * np.pi * f_main * t)
    v_max, v_min = adc_range, -adc_range
    num_levels = 2**bits
    lsb = (v_max - v_min) / num_levels
    
    indices = np.floor((signal - v_min) / lsb)
    indices = np.clip(indices, 0, num_levels - 1)
    quantized = v_min + (indices + 0.5) * lsb
    
    # --- 3. Normalized FFT Processing ---
    n = len(quantized)
    freqs = np.fft.rfftfreq(n, 1/fs)
    fft_mag = np.abs(np.fft.rfft(quantized)) / (n/2)
    
    # Normalize peak to 0 dB
    fft_db_raw = 20 * np.log10(fft_mag + 1e-15)
    max_val = np.max(fft_db_raw)
    fft_db_norm = fft_db_raw - max_val
    
    # --- 4. Formula Calculations ---
    fs_offset = 20 * np.log10(adc_range / signal_amp)
    sqnr_theory = 6.02 * bits + 1.76
    rms_noise_level = fs_offset - sqnr_theory
    proc_gain = 10 * np.log10(n / 2)
    fft_floor_level = rms_noise_level - proc_gain
    
    # --- 5. Plotting ---
    clear_output(wait=True)
    plt.figure(figsize=(12, 8))
    
    plt.plot(freqs, fft_db_norm, color='purple', lw=0.7, label="Quantized Spectrum", alpha=0.6)
    
    # Reference Lines
    plt.axhline(fs_offset, color='blue', linestyle='--', lw=2, 
                label=f"ADC Full Scale (+{fs_offset:.1f} dBc)")
    plt.axhline(rms_noise_level, color='red', linestyle='-', lw=2, 
                label=f"RMS Total Noise Level (6.02N+1.76) = {rms_noise_level:.1f} dBc")
    plt.axhline(fft_floor_level, color='orange', linestyle=':', lw=2.5, 
                label=f"FFT 'Grass' Floor (Proc Gain: -{proc_gain:.1f} dB)")
    plt.axhline(0, color='black', lw=1.5, label="Carrier Peak (0 dBc)")
    
    plt.title(f"Comprehensive ADC Analysis\nBits: {bits} | Samples (M): {n} | Res: {fs/n:.2f} Hz/bin", 
              fontsize=14, fontweight='bold')
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude (dBc)")
    
    plt.ylim(min(fft_floor_level - 15, -135), max(fs_offset + 15, 20))
    plt.xlim(0, fs/2)
    plt.grid(True, which='both', linestyle=':', alpha=0.5)
    plt.legend(loc='lower left', framealpha=0.9, fontsize=10)
    plt.tight_layout()
    plt.show()

# --- UI Controls ---
bits_s = widgets.IntSlider(min=1, max=16, step=1, value=12, description='ADC Bits')
m_slider = widgets.IntSlider(min=8, max=16, step=1, value=12, description='M (2^n)')

ui = VBox([
    HBox([bits_s, m_slider])
])

out = interactive_output(plot_adc_final, {
    'bits': bits_s, 
    'm_power': m_slider
})

display(ui, out)

Output()

## 4. SFDR values

In [60]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, VBox, HBox
from IPython.display import display, clear_output

def plot_sfdr_analysis(bits, signal_amp, adc_range, m_power):
    # --- 1. Parameters ---
    fs = 4096
    M = 2**m_power 
    t = np.arange(M) / fs
    f_main = 997.0 # Using a float for frequency
    
    # --- 2. Signal & Quantization ---
    signal = signal_amp * np.sin(2 * np.pi * f_main * t)
    v_max, v_min = adc_range, -adc_range
    num_levels = 2**bits
    lsb = (v_max - v_min) / num_levels
    
    indices = np.floor((signal - v_min) / lsb)
    indices = np.clip(indices, 0, num_levels - 1)
    quantized = v_min + (indices + 0.5) * lsb
    
    # --- 3. FFT & Normalization ---
    n = len(quantized)
    freqs = np.fft.rfftfreq(n, 1/fs)
    fft_mag = np.abs(np.fft.rfft(quantized)) / (n/2)
    fft_db_fs = 20 * np.log10(fft_mag + 1e-15) # Raw dB relative to 1.0 (scaling needed)
    
    # Scaling to dBFS: 0dB is the maximum possible amplitude (adc_range)
    # Power of full scale sine is (adc_range^2)/2. 
    # We normalize such that a sine with amp=adc_range hits 0 dBFS.
    fft_db_fs = 20 * np.log10(fft_mag / adc_range + 1e-15)
    
    # --- 4. Find SFDR ---
    # Find the carrier index
    carrier_idx = np.argmax(fft_db_fs)
    carrier_level = fft_db_fs[carrier_idx]
    
    # Mask the carrier and its immediate neighbors to find the highest spur
    # We mask +/- 5 bins to avoid picking the main signal sidebands
    fft_no_carrier = fft_db_fs.copy()
    mask_width = 5
    start_mask = max(0, carrier_idx - mask_width)
    end_mask = min(len(fft_db_fs), carrier_idx + mask_width)
    fft_no_carrier[start_mask:end_mask] = -200 # Set to very low value
    
    spur_idx = np.argmax(fft_no_carrier)
    spur_level = fft_db_fs[spur_idx]
    
    # SFDR Calculations
    sfdr_fs = -spur_level            # Relative to Full Scale (0 dBFS)
    sfdr_dbc = carrier_level - spur_level # Relative to Carrier
    
    # --- 5. Plotting ---
    clear_output(wait=True)
    plt.figure(figsize=(12, 7))
    
    plt.plot(freqs, fft_db_fs, color='purple', lw=0.8, alpha=0.7)
    
    # Annotate Carrier and Spur
    plt.plot(freqs[carrier_idx], carrier_level, 'go', label="Carrier")
    plt.plot(freqs[spur_idx], spur_level, 'ro', label="Highest Spur")
    
    # SFDR Visual Bars
    plt.vlines(freqs[spur_idx], spur_level, 0, colors='blue', linestyles='--', 
               label=f'SFDR (Full Scale): {sfdr_fs:.1f} dB')
    plt.vlines(freqs[carrier_idx], spur_level, carrier_level, colors='orange', linestyles=':', 
               lw=2, label=f'SFDR (to Carrier): {sfdr_dbc:.1f} dBc')

    # Formatting
    plt.axhline(0, color='black', lw=1, label="0 dBFS (Full Scale)")
    plt.title(f"SFDR Analysis: {bits} Bits\nSignal Amp: {signal_amp}V | ADC Range: {adc_range}V", 
              fontsize=14, fontweight='bold')
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude (dBFS)")
    plt.ylim(-140, 10)
    plt.grid(True, which='both', linestyle=':', alpha=0.5)
    plt.legend(loc='lower left')
    
    plt.tight_layout()
    plt.show()

# --- UI Controls ---
b_s = widgets.IntSlider(min=1, max=16, step=1, value=8, description='Bits')
a_s = widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=2.0, description='Sig Amp')
r_s = widgets.FloatSlider(min=2.0, max=5.0, step=0.1, value=5.0, description='ADC Range')
m_s = widgets.IntSlider(min=12, max=16, step=1, value=11, description='M (2^n)')

ui = VBox([HBox([b_s, a_s]), HBox([r_s, m_s])])
out = interactive_output(plot_sfdr_analysis, {'bits': b_s, 'signal_amp': a_s, 'adc_range': r_s, 'm_power': m_s})

display(ui, out)

Output()

In [61]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, VBox, HBox, Label
from IPython.display import display, clear_output

def plot_sensitivity_exercise(bits, lna_gain, nf, bw_mhz):
    # --- 1. Thermal Noise Calculation (RF Side) ---
    T = 290 # Kelvin (Room Temp)
    k = 1.38e-23
    bw_hz = bw_mhz * 1e6
    
    # Thermal Noise Floor in dBm: 10*log10(kTB * 1000)
    thermal_noise_dbm = 10 * np.log10(k * T * bw_hz * 1000)
    # Total Noise Floor at ADC input = Thermal + Noise Figure + LNA Gain
    system_noise_floor_dbm = thermal_noise_dbm + nf + lna_gain
    
    # --- 2. ADC Noise Calculation (Digital Side) ---
    # Assume a standard 50-ohm system and 2V peak-to-peak ADC range (Typical)
    adc_vpp = 2.0
    adc_pwr_dbm = 10 # 10 dBm is approx 2Vpp into 50 ohms
    
    # Total Quantization Noise Power relative to Full Scale (dBFS)
    # Formula: -(6.02 * N + 1.76)
    quant_noise_dbfs = -(6.02 * bits + 1.76)
    
    # Convert ADC noise to absolute dBm
    adc_noise_floor_dbm = adc_pwr_dbm + quant_noise_dbfs
    
    # --- 3. Result Visualization ---
    clear_output(wait=True)
    fig, ax = plt.subplots(figsize=(10, 6))
    
    labels = ['RF System Noise', 'ADC Quant. Noise']
    levels = [system_noise_floor_dbm, adc_noise_floor_dbm]
    colors = ['skyblue', 'salmon']
    
    bars = ax.bar(labels, levels, color=colors, alpha=0.8, width=0.5)
    ax.set_ylabel('Power Level (dBm)')
    ax.set_ylim(-130, -20)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Logic check: Is the system noise above the ADC noise?
    margin = system_noise_floor_dbm - adc_noise_floor_dbm
    if margin > 10:
        status = "✅ GOOD: ADC is transparent to thermal noise."
        status_color = 'green'
    elif margin > 0:
        status = "⚠️ WARNING: ADC noise may limit sensitivity."
        status_color = 'orange'
    else:
        status = "❌ FAIL: Receiver is ADC-Limited (Deaf)."
        status_color = 'red'
        
    plt.title(f"Receiver Sensitivity Alignment\n{status}", color=status_color, fontsize=14, fontweight='bold')
    
    # Annotate bars
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.1f} dBm', xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

# --- UI Controls ---
b_s = widgets.IntSlider(min=8, max=16, step=4, value=8, description='ADC Bits')
g_s = widgets.IntSlider(min=0, max=60, step=1, value=20, description='LNA Gain (dB)')
nf_s = widgets.FloatSlider(min=0, max=10, step=0.5, value=3.0, description='Noise Fig (dB)')
bw_s = widgets.FloatSlider(min=0.1, max=20, step=0.5, value=2.0, description='BW (MHz)')

ui = VBox([
    HBox([b_s, g_s]),
    HBox([nf_s, bw_s]),
    Label("Goal: Adjust LNA Gain until 'RF System Noise' is ~10dB above 'ADC Noise'")
])

out = interactive_output(plot_sensitivity_exercise, {'bits': b_s, 'lna_gain': g_s, 'nf': nf_s, 'bw_mhz': bw_s})

display(ui, out)

Output(outputs=({'name': 'stderr', 'text': 'C:\\Users\\bcpmosaj\\AppData\\Local\\Temp\\ipykernel_12724\\249410…